In [1]:
import numpy as np
import pandas as pd
from src.utils import * 

In [2]:
gbsg = pd.read_csv("../GBSG2")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

On détermine $I$ et $J$: 

In [3]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [4]:
# Variables continues
idx_k1 = gbsg.columns[(gbsg.isin([0, 1])==False).any()]

In [5]:
for i in gbsg.columns[(gbsg.isin([0, 1])).all()].to_numpy(): 
    gbsg[i+'_'] = 1-gbsg[i]

#Variables encodées en dummy variables 
idx_j = gbsg.columns[(gbsg.isin([0, 1])).all()]

In [6]:
data_missing = gbsg.mask(np.random.random(size=gbsg.shape)>0.5)

n_missing = data_missing.isna().sum().sum()
n_missing/(data_missing.shape[0]*data_missing.shape[1])

0.500728862973761

In [11]:
class IterativeFAMDImputer(FAMD):
    def __init__(self, data, k1, k2, n_components=2):
        super().__init__(data, k1, k2)
        """Initialisation
        Args:
            n_components (int, optional): _description_. Defaults to 2.
            data (_type_, optional): _description_. Defaults to None.
        """
        self.n_components = n_components
        
    def inital_impute(self, data):
        #Pour les variables continues
        self.sj = self.df_C0.std(axis=0).to_numpy()
        Ximp_C0 = self.df_C0.copy()
        for c in self.df_C0.columns.to_numpy(): 
            Ximp_C0[c] = Ximp_C0[c].fillna(self.df_C0[c].mean())  
        self.df_C0 = Ximp_C0

        # Pour les variables catégorielles 
        self.sqrt_pj = np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]).to_numpy()
        Ximp_categ = self.df_categ.copy() 
        Ximp_categ =Ximp_categ.fillna(np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]))  
        res = Ximp_categ.copy()
        for h in range(3):
            col = [Ximp_categ.columns[h],Ximp_categ.columns[h+3]]
            somme = Ximp_categ[col].sum(axis=1)
            for j in range(self.df.shape[0]):
                res.loc[j, col] = Ximp_categ[col].iloc[j]/somme[j]
        self.df_categ = res
        self.data_concat()
        pass 

    def ponderation_gsbs(self): 
        self.df_C0 = self.df[self.k1] # redefini df_C0 avec le df actuel
        self.sj = self.df_C0.std(axis=0).to_numpy()
        new_df_C0 = self.df_C0 / self.sj
        self.df_C0 = new_df_C0 # redefini df_categ avec le df actuel


        self.df_categ = self.df[self.k2]
        self.sqrt_pj = np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]).to_numpy()
        res = self.df_categ.copy()
        for h in range(3):
            col = [self.df_categ.columns[h],self.df_categ.columns[h+3]]
            somme = self.df_categ[col].sum(axis=1)
            for j in range(self.df.shape[0]):
                res.loc[j, col] = self.df_categ[col].iloc[j]/somme[j]
        self.df_categ = res
        self.data_concat() # mise a jour de df_categ

    
    def impute(self, n_it, tol=1e-4,verbose=False):
        # Initialisation 
        idx_NA = 1- self.df.isna().astype(int).to_numpy() # 1 si obs 0 sinon
        #Initial imputation 
        self.inital_impute(self.df)
        self.data_concat() # on construit les données avec df_C0 et df_categ remplis 

        diff = np.inf
        last_chap = np.inf*np.ones_like(idx_NA)
        i= 0
        while i < n_it and diff > tol: 
            #self.ponderation_gsbs()
            Z_p = self.step3() #Updating D, M already inside
            print(Z_p)
            X_chap = (Z_p + self.M)@np.sqrt(self.D)
            df = pd.DataFrame(idx_NA*(self.df).to_numpy() + + (1- idx_NA)*X_chap.to_numpy(), columns=self.df.columns)
            self.df= df
            diff = ((X_chap - last_chap)**2).mean().mean()
            last_chap = X_chap
            i +=1 
        if i < n_it: 
            print('Converged in', i)
        else: 
              print('Maximum iterations reached')    
        pass 

In [12]:
famd_algo = FAMD(data=gbsg, k1=idx_k1, k2=idx_j)
famd_algo.run_famd()

,pid,age,size,grade,nodes,pgr,er,rfstime,meno,hormon,status,meno_,hormon_,status_
0,-0.002485,-0.009541,-0.083477,-0.259953,-0.278960,0.004512,0.003272,0.007382,-0.000034,-0.000016,-0.000355,0.000049,0.000008,0.000257
1,0.006887,-0.001825,0.232226,1.981576,0.743903,-0.024163,-0.018125,-0.024987,0.000022,-0.000122,0.001144,-0.000033,0.000059,-0.000829
2,-0.000374,-0.046791,-0.011145,1.984400,-0.088813,-0.018028,-0.014038,-0.006155,-0.000119,-0.000271,0.000203,0.000175,0.000131,-0.000147
3,-0.002406,-0.059490,-0.079265,1.990184,-0.322019,-0.016357,-0.012930,-0.000902,-0.000158,-0.000313,-0.000060,0.000234,0.000151,0.000043
4,0.000121,0.006787,0.003877,-0.269323,0.020144,0.002387,0.001865,0.000652,0.000017,0.000038,-0.000018,-0.000026,-0.000018,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,-0.001529,-0.053892,-0.049865,1.982506,-0.221237,-0.017030,-0.013371,-0.003151,-0.000141,-0.000294,0.000053,0.000208,0.000142,-0.000038
682,-0.001978,-0.006013,-0.066485,-0.277420,-0.220379,0.004243,0.003111,0.006128,-0.000023,-0.000004,-0.000292,0.000034,0.000002,0.000211
683,-0.001857,-0.055817,-0.060891,1.977660,-0.258837,-0.016707,-0.013150,-0.002280,-0.000147,-0.000300,0.000010,0.000217,0.000145,-0.000007
684,-0.001815,-0.005360,-0.061008,-0.261534,-0.202048,0.003958,0.002905,0.005648,-0.000020,-0.000002,-0.000268,0.000030,0.000001,0.000194


In [13]:
ifamd = IterativeFAMDImputer(n_components=4, data=data_missing, k1=idx_k1, k2=idx_j)

$\textcolor{red}{\text{REGLER PROBLEME de ponderations! Il y t'il des ponderations entre étapes? On ponderee les categories? Qqchose marche pas car on a des valeurs négatives :(}}$

In [14]:
ifamd.impute(1000)

           pid       age      size     grade     nodes         pgr         er  \
0   -37.320455 -0.230830  0.143969 -0.043278 -0.327689    0.011572  -7.892531   
1    28.687710 -0.026011  0.010198  0.079386  0.300740   -7.426794   0.032622   
2     8.793127  0.081152  0.113582 -0.000586 -0.082372   -7.432185   0.056601   
3    -0.000721 -0.294602  0.035536  0.093433  0.284475   -7.408574  -7.599695   
4    -0.018047  0.122318 -0.007111  0.020720  0.093867   -7.463797   0.184152   
..         ...       ...       ...       ...       ...         ...        ...   
681  -0.009038 -1.367968  0.148456 -0.197676 -0.982303   73.766153  -4.903516   
682   0.005360  2.895090 -2.051524 -0.443980  0.359467   88.190549  81.476173   
683  26.400819 -0.115364  0.119945  0.015357 -0.035781   -0.019961   0.085094   
684  -9.663794 -0.093924  0.022853  0.000447 -0.033438    0.049423  -2.643639   
685  -2.470732  1.119377 -1.760811 -0.597164 -0.619461  160.372306  68.127631   

       rfstime      meno   